In [1]:
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class RFNet(nn.Module) : 
    def __init__(self): 
        super(RFNet, self).__init__() 
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 64, 3) 
        self.fc1 = nn.Linear(64 * 6 * 6 , 64)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x) : 
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1) 
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x 

In [4]:
def train_and_evaluate(model) : 
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(5) : 
        runing_loss = 0.0 
        for i, data in enumerate(trainloader, 0) : 
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            runing_loss += loss.item()
            if i % 200 == 190 : 
                print('[%d, %5d] loss : %.3f' % (epoch + 1, i+1, runing_loss / 200))
                runing_loss = 0.0
                
    print("Finshed Training ....")
     
    correct = 0
    total = 0 
    with torch.no_grad():
        for data in testloader : 
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    print("Accuracy: %.2f %%"% accuracy)

In [5]:
print("RFNet:")
rf_model = RFNet()
train_and_evaluate(rf_model)

print("Number of parameters:")
print("RFNet:", sum(p.numel() for p in rf_model.parameters()))

RFNet:
[1,   191] loss : 2.191
[1,   391] loss : 2.240
[1,   591] loss : 2.097
[2,   191] loss : 1.805
[2,   391] loss : 1.823
[2,   591] loss : 1.763
[3,   191] loss : 1.546
[3,   391] loss : 1.575
[3,   591] loss : 1.532
[4,   191] loss : 1.399
[4,   391] loss : 1.449
[4,   591] loss : 1.425
[5,   191] loss : 1.322
[5,   391] loss : 1.343
[5,   591] loss : 1.355
Finshed Training ....
Accuracy: 52.34 %
Number of parameters:
RFNet: 167562
